In [ ]:
! pip install azure-search-documents --pre
! pip install python-dotenv

In [1]:
# Import required libraries  
import os  
import json  
import openai
import keys
from dotenv import load_dotenv  
from tenacity import retry, wait_random_exponential, stop_after_attempt  
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient  
from azure.search.documents.indexes import SearchIndexClient  
from azure.search.documents.models import Vector  
from azure.search.documents.indexes.models import (  
    SearchIndex,  
    SearchField,  
    SearchFieldDataType,  
    SimpleField,  
    SearchableField,  
    SearchIndex,  
    SemanticConfiguration,  
    PrioritizedFields,  
    SemanticField,  
    SearchField,  
    SemanticSettings,  
    VectorSearch,  
    HnswVectorSearchAlgorithmConfiguration,  
)

import pandas as pd
  
# Configure environment variables  
load_dotenv()  
service_endpoint = "https://aimlexplorationsearch.search.windows.net"
index_name = "rsaf-cognitive-search"
key = "hwioAbEseT8hs3Pv98fREcnMpoqPQnp9FdR0OwoPpmAzSeCG2vMW"

# key = "wGvVdInupyqUH2YraI1mpZmviDLXjh3v3i2zZetKAnAzSeCZGrhS"
# service_endpoint = "https://rsaf-cognitive-search-service-us.search.windows.net"

openai.api_type = "azure"
openai.api_key = "0e576c031e5446e19cc6d866ad3d9f3f"
openai.api_base = "https://raid-ses-openai.openai.azure.com/"
openai.api_version = "2023-05-15"
credential = AzureKeyCredential(key)

In [2]:
client_index = SearchIndexClient(endpoint=service_endpoint, credential=credential)

In [36]:
client_index.delete_index("rsaf-cognitive-search")

In [3]:
for i in client_index.list_indexes():
    print(i.name)

rsaf-cognitive-search-new
rsaf-cognitive-search


In [5]:
from azure.search.documents.models import (
    QueryAnswerType,
    QueryCaptionType,
    QueryLanguage,
    QueryType  
)

search_client = SearchClient(endpoint=service_endpoint, index_name="rsaf-cognitive-search-new", credential=credential)

all_docs = search_client.search(search_text="fenestron", query_type=QueryType.SEMANTIC, query_language=QueryLanguage.EN_US, semantic_configuration_name='default',top=6)

pd.DataFrame(all_docs)

,id,hyperlink,filename,page,li_jsonMetadata,content,embedding,li_doc_id,total_pages,@search.score,@search.reranker_score,@search.highlights,@search.captions
0,ae3dba83-3140-4d00-a6e9-530cc2603c18,#page=13,EEM_FINAL_caa201221.pdf,13,"{""total_pages"": 319, ""page"": 13, ""filename"": ""...",OFFICIAL (CLOSED)\nANNEX A\nTO EEM CHAPTER 1\n...,"[-0.01214574, -0.0114063155, -0.021183157, -0....",2fd052fc-7b01-4cb0-aba3-1d410b9db831,319,5.566793,2.744638,None,None
1,abfe7577-676f-488a-a179-2e508602d388,#page=15,EEM_FINAL_caa201221.pdf,15,"{""total_pages"": 319, ""page"": 15, ""filename"": ""...",The fenestron is different in that the aerofoi...,"[-0.013530294, 0.010190538, -0.017769216, -0.0...",2fd052fc-7b01-4cb0-aba3-1d410b9db831,319,5.447430,2.561581,None,None
2,89c19010-9c0d-473c-a309-b8f6210d6372,#page=14,EEM_FINAL_caa201221.pdf,14,"{""total_pages"": 319, ""page"": 14, ""filename"": ""...",OFFICIAL (CLOSED)\nhas seven stator vanes that...,"[-0.01354955, 0.007836729, -0.018295048, -0.00...",2fd052fc-7b01-4cb0-aba3-1d410b9db831,319,5.186345,2.349035,None,None
3,98c3258e-4022-4e1e-baad-8c400fe4146a,#page=23,EEM_FINAL_caa201221.pdf,23,"{""total_pages"": 319, ""page"": 23, ""filename"": ""...",OFFICIAL (CLOSED)\noccurs when the collective ...,"[-0.023863992, -0.01298642, 0.01082315, -0.020...",2fd052fc-7b01-4cb0-aba3-1d410b9db831,319,3.127924,2.157839,None,None
4,5790a4d3-7bc8-4e03-bfc1-ac71dc9b427e,#page=237,EEM_FINAL_caa201221.pdf,237,"{""total_pages"": 319, ""page"": 237, ""filename"": ...",OFFICIAL (CLOSED)\nFigure 19-3: Tac Right\n920...,"[0.0019079837, -0.0067284326, -0.011812846, -0...",2fd052fc-7b01-4cb0-aba3-1d410b9db831,319,5.915940,1.985217,None,None


In [4]:
search_client = SearchClient(endpoint=service_endpoint, index_name="rsaf-cognitive-search", credential=credential)

all_docs = search_client.search(search_text="*", top=50)

In [77]:
for i in all_docs:
    text = "Filename: " + i["filename"] + "\n" + "Hyperlink: " + i["hyperlink"] + "\n" + "Page Number: " + str(i["page"]) + "\n" + "Content: " + i["content"] + "\n-----------------------"

In [8]:
doc = search_client.search(search_text="*", filter="li_doc_id eq '2fd052fc-7b01-4cb0-aba3-1d410b9db831'")

pd.DataFrame(doc)

,id,hyperlink,filename,page,li_jsonMetadata,content,embedding,li_doc_id,total_pages,@search.score,@search.reranker_score,@search.highlights,@search.captions
0,ca968e74-befb-4348-a36d-7707078ae735,#page=15,EEM_FINAL_caa201221.pdf,15,"{""total_pages"": 319, ""page"": 15, ""filename"": ""...",OFFICIAL (CLOSED)\n1A.4 Horizontal Stabilizer....,"[-0.02099528, 0.011131816, 0.003132088, -0.004...",2fd052fc-7b01-4cb0-aba3-1d410b9db831,319,1.0,None,None,None
1,0b13303c-6925-4530-9f12-b4405936b3f2,#page=22,EEM_FINAL_caa201221.pdf,22,"{""total_pages"": 319, ""page"": 22, ""filename"": ""...",OFFICIAL (CLOSED)\nsame direction. If pedals a...,"[-0.020845745, -0.027577184, 0.0050621503, -0....",2fd052fc-7b01-4cb0-aba3-1d410b9db831,319,1.0,None,None,None
2,f4baac03-c570-4e47-9792-c2527baec243,#page=26,EEM_FINAL_caa201221.pdf,26,"{""total_pages"": 319, ""page"": 26, ""filename"": ""...",OFFICIAL (CLOSED)\nPART A\nCHAPTER 3\nSTRAIGHT...,"[-0.004480087, -0.013107533, -0.00070116896, -...",2fd052fc-7b01-4cb0-aba3-1d410b9db831,319,1.0,None,None,None
3,a77fcb78-393a-438b-8412-4bbc9419508c,#page=36,EEM_FINAL_caa201221.pdf,36,"{""total_pages"": 319, ""page"": 36, ""filename"": ""...","To level off, anticipate the height\nrequired ...","[-0.004218373, 0.004316069, 0.0008740302, -0.0...",2fd052fc-7b01-4cb0-aba3-1d410b9db831,319,1.0,None,None,None
4,53f93dfd-96a6-4618-a893-6a7a3b4a6f0c,#page=37,EEM_FINAL_caa201221.pdf,37,"{""total_pages"": 319, ""page"": 37, ""filename"": ""...",OFFICIAL (CLOSED)\na.\nPower. Catering for 10%...,"[0.01250267, -0.007914661, 0.02327271, -0.0088...",2fd052fc-7b01-4cb0-aba3-1d410b9db831,319,1.0,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
425,5d9daa3f-318b-4a09-a239-44570334899f,#page=291,EEM_FINAL_caa201221.pdf,291,"{""total_pages"": 319, ""page"": 291, ""filename"": ...","OFFICIAL (CLOSED)\nover water, snow or desert ...","[0.011870887, 0.013876565, 0.00974963, -0.0162...",2fd052fc-7b01-4cb0-aba3-1d410b9db831,319,1.0,None,None,None
426,7aabbafb-ccf4-411e-96b0-dddca810793b,#page=293,EEM_FINAL_caa201221.pdf,293,"{""total_pages"": 319, ""page"": 293, ""filename"": ...",OFFICIAL (CLOSED)\nillusion arises when these ...,"[-0.005931039, -0.00470536, 0.0058375546, -0.0...",2fd052fc-7b01-4cb0-aba3-1d410b9db831,319,1.0,None,None,None
427,160023f9-c561-4f31-a2d1-72bdc98cf86e,#page=303,EEM_FINAL_caa201221.pdf,303,"{""total_pages"": 319, ""page"": 303, ""filename"": ...","At approximately 15ft, raise the collective to...","[-0.019286081, -0.018516287, -0.0025980535, -0...",2fd052fc-7b01-4cb0-aba3-1d410b9db831,319,1.0,None,None,None
428,72af070a-0cfc-49db-8eb4-fe0a9a8e5661,#page=308,EEM_FINAL_caa201221.pdf,308,"{""total_pages"": 319, ""page"": 308, ""filename"": ...",TRANSMISSION EMERGENCIES\n22.33\nCOMPLETE LOSS...,"[-0.013492667, -0.019651769, -0.0020382395, -0...",2fd052fc-7b01-4cb0-aba3-1d410b9db831,319,1.0,None,None,None


In [20]:
DOCUMENT = [{"id" : "2ac5e300-d404-4b13-bde5-6d723a3cb7da"}]

search_client.delete_documents(documents=DOCUMENT)

### Vector Search

In [12]:
def generate_embeddings(text):
    response = openai.Embedding.create(
        input=text, engine="swiftfaq-ada002")
    embeddings = response['data'][0]['embedding']
    return embeddings

In [30]:
search_client = SearchClient(endpoint=service_endpoint, index_name="chat-demo", credential=credential)

pd.DataFrame(search_client.search("autorotation", top=3))

,embedding,id,li_jsonMetadata,li_doc_id,page_label,content,@search.score,@search.reranker_score,@search.highlights,@search.captions
0,"[-0.021303589, -0.008744429, -0.020742599, -0....",21c6d4c3-6106-4f3d-b628-73e129507b7c,"{""page_label"": ""111"", ""file_name"": ""EEM FINAL ...",2da13a2e-fbc3-4d15-b8fa-6e1517074460,111,OFFICIAL (CLOSED) \n11-1 \nOFFICIAL (CLOSED)...,3.408726,None,None,None
1,"[-0.017312882, -0.0033271702, -0.012544677, -0...",2f71f668-85dc-4155-9b57-104767b0c28b,"{""page_label"": ""42"", ""file_name"": ""EEM FINAL c...",87546a10-0794-45fa-a8cf-46919ebe14ce,42,OFFICIAL (CLOSED) \n5-1 \nOFFICIAL (CLOSED) ...,3.335187,None,None,None
2,"[-0.022873871, -0.013425363, -0.0023482218, -0...",2282ddc7-7b38-4f25-815e-dc8d3f323e1d,"{""page_label"": ""116"", ""file_name"": ""EEM FINAL ...",bd14dd00-8091-4885-9421-86003faa860b,116,OFFICIAL (CLOSED) \n11-6 \nOFFICIAL (CLOSED)...,3.253138,None,None,None
